In [3]:
# for observability and monitoring
from langsmith import traceable
from langsmith.wrappers import  wrap_openai

# to load the model
from langchain_community.llms import Ollama

# to design the prompt template
from langchain_core.prompts import ChatPromptTemplate

# to load the document
from langchain_community.document_loaders import WebBaseLoader, PyPDFLoader, DirectoryLoader

# to convert document contents into embeddings
from langchain_community.embeddings import OllamaEmbeddings

# save our embeddings into a vector and create a vectorstore retriver object
from langchain_community.vectorstores import FAISS

# document content splitter
from langchain_text_splitters import RecursiveCharacterTextSplitter




In [18]:
import os

In [25]:
# load enviroment variales
from dotenv import load_dotenv
load_dotenv()

True

In [10]:
# load a sample document from local file system
# loader = PyPDFLoader('/Users/eugene/Personal_Projects/Real_ML_Project/langchain_agent_app/National Artificial Intelligence Strategy.pdf')
loader = DirectoryLoader('/Users/eugene/Personal_Projects/Real_ML_Project/langchain_agent_app/files/',
                         show_progress = True,
                         sample_size = 1,
                         use_multithreading = True)
docs = loader.load()

# # load a sample website
# loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")
# docs = loader.load()




100%|██████████| 1/1 [07:15<00:00, 435.07s/it]
/Users/eugene/Personal_Projects/Real_ML_Project/langchain_agent_app/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]



100%|██████████| 1/1 [01:41<00:00, 101.90s/it]


In [11]:
# visualize the contents of our document
docs[0:5]

[Document(metadata={'source': '/Users/eugene/Personal_Projects/Real_ML_Project/langchain_agent_app/files/National Artificial Intelligence Strategy.pdf'}, page_content='DRAFT\n\nA U G U S T , 2 0 2 4\n\nNATIONAL ARTIFICIAL INTELLIG EN CE STR AT EGY 2 024\n\nAcknowledgements\n\nThe Federal Ministry of Communication, Innovation and Digital Economy (FMCIDE)\n\nwould like to acknowledge the contributions of all stakeholders who provided data,\n\nDRAFT\n\nparticipated in workshops and interviews, and reviewed and provided comments as this\n\nNational AI Strategy was drafted and ﬁnalised. In particular, the Ministry appreciates the\n\nsupport of the Lagos Business School, Data Science Nigeria, the National Information\n\nTechnology Development Agency (NITDA), Nigerian Communications Commission (NCC),\n\nGalaxy Backbone Limited (GBB) and other stakeholders that assisted in no small measure to\n\nproduce this National Artiﬁcial Intelligence Strategy.\n\n2.\n\nNATIONAL ARTIFICIAL INTELLIG EN CE 

In [12]:
embedding_obj = OllamaEmbeddings(model="llama3",
                                 show_progress_bar=True,)

In [15]:
# creating our retriever object

# performing the text splitting
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, chunk_overlap=100
)

splitted_doc = text_splitter.split_documents(docs)
# vector = FAISS.from_documents(splitted_doc, embedding_obj)
# retriever = vector.as_retriever()

In [23]:
from langchain_community.vectorstores import Qdrant
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

In [33]:
# retrieve api key and cluster url for our qdrant vector database
api_key  = os.getenv("QDRANT_API_KEY")
url = os.getenv("QDRANT_CLOUD_CLUSTER_URL")

#setup qdrant client and collection objects

client = QdrantClient(
    url=url,
    prefer_grpc=True,
    api_key=api_key,
)

client.create_collection(
    collection_name="my_documents",
    vectors_config=VectorParams(size=4096, distance=Distance.COSINE),
)

True

In [34]:
# creating the vector store object
vector_store = QdrantVectorStore(
    client = client,
    collection_name = "my_documents",
    embedding=embedding_obj,
)

In [49]:
# upload our document to the vectorstore
vector_store.add_documents(splitted_doc)

['0a4e8465f9464e00bef54542c2132a3c',
 '8015a4dab9b943cfbac32c0e89cf3639',
 '0f9fdcef73e84fca9342827fe5518893',
 'fa4af75703dc4de291542f0a5223393a',
 '6f2b7f5f46e44380a6ae4453add10212',
 'e5e9accd044745fcb8e8a5904a2779d7',
 '451b884df2cc4d6490949333b7380f02',
 '6bd0d911b3704ee4862d098d91aa6dde',
 '22619c1074554bfcb790ef92b2d6b07e',
 '28b9831318fc494e8b98095e180b5be5',
 'cde084efd0784e20b109bf281ef54b6e',
 '1019aa30284148a48c117568b64d7a42',
 '81fc9b79dfa143c5a28f93cb24dfd722',
 '5843afd550f449348528f3cef2d5f6ae',
 'd922a05eb77b46329926c4b055895a26',
 'f08cb38a18ff4443ac25e71a42bc1df5',
 'a5f3501ee00d418797ced811e144e411',
 'd02399103cd3496c813a8b7f0445af74',
 '083749be7fb14268929a83de1ff0291f',
 '462961ec2e104f5282c78ac1ed75d5ad',
 '702c56cd5484421584e43c17ad98ea42',
 '672fa5fc3ff14d5f83c5f53272184bee',
 'a880858490004c1e9d9e83acf60394e9',
 '295832ae50174d3fbfec2a09c9e9ec83',
 'c01d681ab300432398935cc4c712b05b',
 '41e0a3ac60c44a46bb2e5087e8eed6b7',
 'c4913723b8f54f1d8b8c248d97cacbe2',
 

In [59]:
query = 'summarize the document key points'
response = vector_store.similarity_search_with_score(query)

In [60]:
response

[(Document(metadata={'source': '/Users/eugene/Personal_Projects/Real_ML_Project/langchain_agent_app/files/National Artificial Intelligence Strategy.pdf', '_id': '1d8813c7-8c27-4409-b79c-da5245d90b41', '_collection_name': 'my_documents'}, page_content='42.\n\nNATIONAL ARTIFICIAL INTELLIG EN CE STR AT EGY 2 024\n\n1.4.6.2. Box: Regulatory Acts and Initiatives Supporting an AI Ecosystem'),
  0.6071240305900574),
 (Document(metadata={'source': '/Users/eugene/Personal_Projects/Real_ML_Project/langchain_agent_app/files/National Artificial Intelligence Strategy.pdf', '_id': '6d40db14-0dc5-4e48-8f83-a8278004cc3c', '_collection_name': 'my_documents'}, page_content='22.\n\nNATIONAL ARTIFICIAL INTELLIG EN CE STR AT EGY 2 024\n\n1.1.5.2. Mitigating AI Risks'),
  0.6064786314964294),
 (Document(metadata={'source': '/Users/eugene/Personal_Projects/Real_ML_Project/langchain_agent_app/files/National Artificial Intelligence Strategy.pdf', '_id': '5a1b7902-3c99-4bf3-b16d-df84d6562543', '_collection_name

In [61]:
answer, score = response[0]
print(f"Answer: {answer.page_content}")
print(f"Relevance Score: {score}")

Answer: 42.

NATIONAL ARTIFICIAL INTELLIG EN CE STR AT EGY 2 024

1.4.6.2. Box: Regulatory Acts and Initiatives Supporting an AI Ecosystem
Relevance Score: 0.6071240305900574


In [13]:
# Defining the two tools for our agent - a retriver and a search 
# a retriver will let us easily answer questions about the documents
# a search will let us find the most relevant document information for a given question
from langchain.tools.retriever import create_retriever_tool
from langchain.tools.tavily_search import TavilySearchResults

retriever_tool = create_retriever_tool(
    retriever,
    name="retriever",
    description="Searches and returns documents based on their content and metadata.",
)

# setting up the search tool
search = TavilySearchResults()

# creating the final tool to used by our agent
tool = [retriever_tool, search]

In [14]:
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor
from langchain import hub
from langchain_ollama import ChatOllama

In [17]:
# creating a dummy prompt
prompt = hub.pull("hwchase17/openai-functions-agent")

llm =  ChatOllama(model="llama3", 
              temperature=0.8,
              num_predict = 256, )

agent = create_openai_functions_agent(
    llm=llm,
    prompt=prompt,
    tools=tool,
)

agent_executor = AgentExecutor(
                    agent=agent,
                    tools = tool,
                    verbose=True
                )

In [18]:
agent_executor.invoke({"input": "what is the weather in liverpool, England?"})



> Entering new AgentExecutor chain...
Liverpool, England!

As of now (please note that weather can change quickly), the current weather conditions in Liverpool are:

* Temperature: 12°C (54°F)
* Conditions: Overcast with light rain showers
* Wind Speed: 20 km/h (12 mph) from the west
* Precipitation Probability: 80%

Please keep in mind that these conditions may be subject to change, and it's always a good idea to check for updates before planning your day.

Would you like me to check the weather forecast for a specific time or date?

> Finished chain.


{'input': 'what is the weather in liverpool, England?',
 'output': "Liverpool, England!\n\nAs of now (please note that weather can change quickly), the current weather conditions in Liverpool are:\n\n* Temperature: 12°C (54°F)\n* Conditions: Overcast with light rain showers\n* Wind Speed: 20 km/h (12 mph) from the west\n* Precipitation Probability: 80%\n\nPlease keep in mind that these conditions may be subject to change, and it's always a good idea to check for updates before planning your day.\n\nWould you like me to check the weather forecast for a specific time or date?"}